In [1]:
!pip install qiskit qiskit_aer

In [3]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')

Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [5]:
# Task 1: change secret string and verify
def bv_oracle(qc, inputs, ancilla, s):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)                # H on all qubits (inputs + ancilla)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    # measure into classical bits in reversed order so printed string equals s
    qc.measure(list(reversed(inputs)), creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    print("Counts:", counts)
    plot_histogram(counts)
    plt.show()
    measured = max(counts, key=counts.get)
    print("Measured (most frequent):", measured)
    return measured

# Example: change secret string here
if __name__ == "__main__":
    s = "0110"   # change this string to test
    print("Secret s =", s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    print("Match:", measured == s)

Secret s = 0110
     ┌───┐┌───┐          ┌─┐                
q_0: ┤ H ├┤ H ├──────────┤M├────────────────
     ├───┤└───┘     ┌───┐└╥┘          ┌─┐   
q_1: ┤ H ├───────■──┤ H ├─╫───────────┤M├───
     ├───┤       │  └───┘ ║      ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────────╫───■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │   ┌─┐  ║   │  └───┘ ║ └╥┘
q_3: ┤ H ├┤ H ├──┼───┤M├──╫───┼────────╫──╫─
     ├───┤├───┤┌─┴─┐ └╥┘  ║ ┌─┴─┐      ║  ║ 
q_4: ┤ X ├┤ H ├┤ X ├──╫───╫─┤ X ├──────╫──╫─
     └───┘└───┘└───┘  ║   ║ └───┘      ║  ║ 
c: 4/═════════════════╩═══╩════════════╩══╩═
                      0   3            2  1 
Counts: {'0110': 1024}
Measured (most frequent): 0110
Match: True


In [7]:
# Task 2: oracle with constant b
def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    # apply CNOTs for s·x
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    # apply constant b by flipping ancilla if b==1
    if b == '1' or b == 1:
        qc.x(ancilla)

def bernstein_vazirani_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg_in = ClassicalRegister(n, 'c')   # input measurement
    creg_anc = ClassicalRegister(1, 'a')  # ancilla measurement (optional)
    qc = QuantumCircuit(qreg, creg_in, creg_anc)

    inputs = list(range(n))
    ancilla = n

    # prepare ancilla to |1> then H -> |-> (phase-kickback form)
    qc.x(ancilla)
    qc.h(ancilla)

    # Hadamard on inputs
    for q in inputs:
        qc.h(q)

    # oracle: f(x) = s·x ⊕ b
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    # Hadamard on inputs again
    for q in inputs:
        qc.h(q)

    # measure inputs in reversed order to match s
    qc.measure(list(reversed(inputs)), creg_in)

    # to inspect ancilla computational value after oracle: uncompute ancilla's H and X
    # (if you want to see 'b' reflected on ancilla, unapply the initial H and X)
    qc.h(ancilla)
    qc.x(ancilla)
    qc.measure(ancilla, creg_anc[0])

    return qc

# Run demonstration
if __name__ == "__main__":
    s = "1011"
    b = "1"   # try "0" and "1"
    qc = bernstein_vazirani_with_b(s, b)
    print(qc.draw(fold=-1))

    sim = AerSimulator()
    job = sim.run(transpile(qc, sim), shots=1024)
    counts = job.result().get_counts()
    print("Counts:", counts)
    plot_histogram(counts)
    plt.show()

     ┌───┐          ┌───┐          ┌─┐                        
q_0: ┤ H ├───────■──┤ H ├──────────┤M├────────────────────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘                        
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫─────────────────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐                
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├────────────────
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐             
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├─────────────
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘┌───┐┌───┐┌─┐
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ X ├─╫──╫─┤ H ├┤ X ├┤M├
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ └───┘└───┘└╥┘
c: 4/═════════════════╩═════════════╩═══════╩══╩════════════╬═
                      2             3       1  0            ║ 
a: 1/═══════════════════════════════════════════════════════╩═
                                                            0 
Counts: {'0 1011': 1024}


In [9]:
!pip install qiskit-aer --quiet

In [11]:
# Task 4: simulate noise model (fixed for Qiskit 2.x / Aer 0.17+)

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator, noise
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Define BV oracle and circuit
def bv_oracle(qc, inputs, ancilla, s):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(list(reversed(inputs)), creg)
    return qc

# Build circuit
s = "1011"
qc = bernstein_vazirani_circuit(s)

# Create a simple depolarizing noise model
noise_model = noise.NoiseModel()
p1 = 0.01  # 1-qubit depolarizing error probability
p2 = 0.05  # 2-qubit depolarizing error probability

error1 = noise.depolarizing_error(p1, 1)
error2 = noise.depolarizing_error(p2, 2)

noise_model.add_all_qubit_quantum_error(error1, ['h', 'x', 'id'])
noise_model.add_all_qubit_quantum_error(error2, ['cx'])

# Simulators
ideal_sim = AerSimulator()
noisy_sim = AerSimulator(noise_model=noise_model)

# Run both
tqc_ideal = transpile(qc, ideal_sim)
tqc_noisy = transpile(qc, noisy_sim)

result_ideal = ideal_sim.run(tqc_ideal, shots=2048).result()
result_noisy = noisy_sim.run(tqc_noisy, shots=2048).result()

counts_ideal = result_ideal.get_counts()
counts_noisy = result_noisy.get_counts()

# Display
print("Ideal counts:", counts_ideal)
print("Noisy counts:", counts_noisy)

plot_histogram([counts_ideal, counts_noisy],
               legend=['Ideal', 'Noisy'],
               title='Bernstein–Vazirani under Noise')
plt.show()

Ideal counts: {'1011': 2048}
Noisy counts: {'0010': 2, '1011': 1823, '0011': 35, '0001': 2, '1001': 37, '1010': 81, '0000': 32, '1000': 36}


In [13]:
#Task 5:reate a notebook that explains each step with visualizations and markdown.
!pip install qiskit qiskit-aer --quiet
print("✅ Qiskit and Aer installed.")

✅ Qiskit and Aer installed.


In [15]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator, noise
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

print("Environment ready.")

Environment ready.


# Bernstein–Vazirani Algorithm in Qiskit (Task 5)
### Goals
- Implement the Bernstein–Vazirani algorithm
- Explore the hidden bit string `s`
- Add a constant bit `b`
- Simulate noise effects
- Interpret and visualize results

In [19]:
def bv_oracle(qc, inputs, ancilla, s):
    """Implements f(x) = s·x."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(list(reversed(inputs)), creg)
    return qc

In [21]:
#Run the Basic Algorithm (Task 1)
def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    measured = max(counts, key=counts.get)
    print("Measured bitstring:", measured)
    return measured

s = "1011"   # Change this string to test different secrets
qc = bernstein_vazirani_circuit(s)
print(qc.draw(fold=-1))
measured = run_bv(qc)
print("Match:", measured == s)

     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      2             3       1  0 
Measured bitstring: 1011
Match: True


In [23]:
#Add Constant Bit b (Task 2)
def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg_in = ClassicalRegister(n, 'c')
    creg_anc = ClassicalRegister(1, 'a')
    qc = QuantumCircuit(qreg, creg_in, creg_anc)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(ancilla)
    for q in inputs:
        qc.h(q)

    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    for q in inputs:
        qc.h(q)
    qc.measure(list(reversed(inputs)), creg_in)

    qc.h(ancilla)
    qc.x(ancilla)
    qc.measure(ancilla, creg_anc[0])
    return qc

s, b = "1011", "1"
qc = bernstein_vazirani_with_b(s, b)
print(qc.draw(fold=-1))
sim = AerSimulator()
counts = sim.run(transpile(qc, sim), shots=1024).result().get_counts()
print("Counts:", counts)
plot_histogram(counts)
plt.show()

     ┌───┐          ┌───┐          ┌─┐                        
q_0: ┤ H ├───────■──┤ H ├──────────┤M├────────────────────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘                        
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫─────────────────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐                
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├────────────────
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐             
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├─────────────
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘┌───┐┌───┐┌─┐
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ X ├─╫──╫─┤ H ├┤ X ├┤M├
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ └───┘└───┘└╥┘
c: 4/═════════════════╩═════════════╩═══════╩══╩════════════╬═
                      2             3       1  0            ║ 
a: 1/═══════════════════════════════════════════════════════╩═
                                                            0 
Counts: {'0 1011': 1024}


In [33]:
print("Ideal counts:", counts_ideal)
print("Noisy counts:", counts_noisy)

Ideal counts: {'1011': 2048}
Noisy counts: {'0000': 48, '1000': 42, '1011': 1794, '0001': 3, '1001': 36, '1010': 77, '0011': 48}


In [39]:
# Noise Simulation (Task 4)
from qiskit_aer import AerSimulator, noise
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Create depolarizing noise model
noise_model = noise.NoiseModel()
error1 = noise.depolarizing_error(0.01, 1)
error2 = noise.depolarizing_error(0.05, 2)
noise_model.add_all_qubit_quantum_error(error1, ['h', 'x', 'id'])
noise_model.add_all_qubit_quantum_error(error2, ['cx'])

# Build BV circuit
s = "1011"
qc = bernstein_vazirani_circuit(s)

# Simulators
ideal_sim = AerSimulator()
noisy_sim = AerSimulator(noise_model=noise_model)

# Run simulations
tqc_ideal = transpile(qc, ideal_sim)
tqc_noisy = transpile(qc, noisy_sim)

res_ideal = ideal_sim.run(tqc_ideal, shots=2048).result()
res_noisy = noisy_sim.run(tqc_noisy, shots=2048).result()

counts_ideal = res_ideal.get_counts()
counts_noisy = res_noisy.get_counts()

# Print to confirm
print("Ideal counts:", counts_ideal)
print("Noisy counts:", counts_noisy)

# Plot comparison
fig = plot_histogram(
    [counts_ideal, counts_noisy],
    legend=['Ideal', 'Noisy'],
    title='Effect of Noise on BV Algorithm'
)
plt.show(fig)

Ideal counts: {'1011': 2048}
Noisy counts: {'0010': 2, '1011': 1791, '0001': 3, '1010': 86, '1001': 36, '0000': 41, '1000': 40, '0011': 49}


## Summary
- The Bernstein–Vazirani algorithm finds a hidden bit string `s` in one oracle query.  
- Adding a constant `b` affects only the ancilla qubit (global phase).  
- Noise reduces accuracy, but small depolarizing errors still allow correct recovery of `s`.

**Next experiments**
- Try longer bitstrings.
- Test on real IBM hardware using `qiskit_ibm_runtime`.
- Sweep noise probabilities and plot success rate.